In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
def reduce_mem_usage(df):
    """Reduce memory usage of DataFrame by downcasting numeric types"""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type != 'category' and col != 'TransactionID' and 'ID' not in col:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    print(f'Decreased by {100 * (start_mem - end_mem) / start_mem:.1f}%')
    
    return df

In [3]:
!pip install dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 5.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.


In [4]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 50.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 79.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00


In [5]:
import mlflow
import dagshub
dagshub.init(repo_owner='dshan21', repo_name='ML_ASS_2', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=67d3b2d9-b28a-4da8-86e3-3e579b9e337c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=06f08c2f81f1c229144915a8bfe8670ed703452a5e4edf7cabdd26d61d08316f




Output()

Accessing as dshan21

Initialized MLflow to track repo "dshan21/ML_ASS_2"

Repository dshan21/ML_ASS_2 initialized!

In [6]:
identity_train_file_path = "/kaggle/input/ieee-fraud-detection/train_identity.csv"
identity_df = pd.read_csv(identity_train_file_path)
transaction_train_file_path = "/kaggle/input/ieee-fraud-detection/train_transaction.csv"
transaction_df = pd.read_csv(transaction_train_file_path)

In [7]:
mlflow.set_experiment("RandomForest_Training")

<Experiment: artifact_location='mlflow-artifacts:/cdb96b3fd8e24f6493175a1b005a8151', creation_time=1745675260978, experiment_id='2', last_update_time=1745675260978, lifecycle_stage='active', name='RandomForest_Training', tags={}>

# FEATURE CLEANING

In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
merged_df = None
y = None

with mlflow.start_run(run_name="RandomForest_Cleaning"):
    missing_cutoff = 0.9
    merged_df = transaction_df.merge(identity_df, on='TransactionID', how='left')
    # merged_df = reduce_mem_usage(merged_df)
    # print(f"Merged data shape: {merged_df.shape}")
    
    y = merged_df['isFraud'].copy()
    merged_df.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)
    
    missing_rate = merged_df.isnull().mean()
    high_missing_cols = missing_rate[missing_rate > missing_cutoff].index.tolist()
    mlflow.log_param("high_missing_threshold", missing_cutoff)
    
    high_missing_df = pd.DataFrame({
        'column_name': high_missing_cols,
        'missing_rate': [missing_rate[col] for col in high_missing_cols]
    }).sort_values('missing_rate', ascending=False)
    
    high_missing_file = "high_missing_columns.csv"
    high_missing_df.to_csv(high_missing_file, index=False)
    
    mlflow.log_artifact(high_missing_file)
    
    merged_df.drop(high_missing_cols, axis=1, inplace=True)
    # merged_df = reduce_mem_usage(merged_df)
    mlflow.log_param("removed_columns", len(high_missing_cols))
    
    categorical_cols = merged_df.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = merged_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    # print(categorical_cols)
    # print(numerical_cols)
    
    for col in numerical_cols:
        if merged_df[col].isnull().sum() > 0:
            merged_df[col] = merged_df[col].fillna(merged_df[col].median())
    
    for col in categorical_cols:
        if merged_df[col].isnull().sum() > 0:
            merged_df[col] = merged_df[col].fillna('Unknown')
            
    categorical_encoder = LabelEncoder()      
    mlflow.log_param("categorical_col_repl_method", categorical_encoder)
    if isinstance(categorical_encoder, LabelEncoder):
        for col in categorical_cols:
            merged_df[col] = categorical_encoder.fit_transform(merged_df[col].astype(str)) 
    elif isinstance(categorical_encoder, OneHotEncoder):
        for col in categorical_cols:
            encoded_array = categorical_encoder.fit_transform(merged_df[col].astype(str).values.reshape(-1, 1))
            encoded_cols = categorical_encoder.get_feature_names_out([col])
            encoded_df = pd.DataFrame(encoded_array.toarray(), columns=encoded_cols, index=merged_df.index)
            
            merged_df.drop(columns=[col], inplace=True)
            merged_df = pd.concat([merged_df, encoded_df], axis=1)
            
    mlflow.log_param("final_feature_count",  merged_df.shape[1])
    # merged_df = reduce_mem_usage(merged_df)

🏃 View run RandomForest_Cleaning at: https://dagshub.com/dshan21/ML_ASS_2.mlflow/#/experiments/2/runs/630fc3ede9894972b6fa8d87577ac7eb
🧪 View experiment at: https://dagshub.com/dshan21/ML_ASS_2.mlflow/#/experiments/2


# FEATURE ENGINEERING

In [9]:
newframe = merged_df.copy()
merged_df = newframe

merged_df['transaction_day'] = merged_df['TransactionDT'] // (24 * 60 * 60)
merged_df['transaction_hour'] = (merged_df['TransactionDT'] % (24 * 60 * 60)) // (60 * 60)
merged_df['transaction_minute'] = ((merged_df['TransactionDT'] % (24 * 60 * 60)) % (60 * 60)) // 60
merged_df['transaction_second'] = merged_df['TransactionDT'] % 60

merged_df['day_of_week'] = merged_df['transaction_day'] % 7
merged_df['is_weekend'] = merged_df['day_of_week'].isin([0, 6]).astype(int)

merged_df.drop('TransactionDT', axis=1, inplace=True)

merged_df['amount_log'] = np.log1p(merged_df['TransactionAmt'])
merged_df['amount_decimal'] = merged_df['TransactionAmt'] - np.floor(merged_df['TransactionAmt'])

# FEATURE SELECTION

In [10]:
# from sklearn.model_selection import train_test_split
# X_cleaned = merged_df
# X_train, X_val, y_train, y_val = train_test_split(
#     X_cleaned, y, test_size=0.2, random_state=42, stratify=y
# )

# print("NaN values in X_train:", X_train.isna().sum().sum())

# X_train = X_train.fillna(X_train.median())
# X_val = X_val.fillna(X_val.median())

NaN values in X_train: 0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

X_selected = None
X_cleaned = merged_df
with mlflow.start_run(run_name="RandomForest_Feature_Selection"):
 
    X_train, X_val, y_train, y_val = train_test_split(
        X_cleaned, y, test_size=0.2, random_state=42, stratify=y
    )
    
    rf_feat = RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    )
    
    rf_feat.fit(X_train, y_train)
    
    feature_importances = rf_feat.feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': X_cleaned.columns,
        'Importance': feature_importances
    }).sort_values('Importance', ascending=False)
    
    top_n_features = 200
    top_features = feature_importance_df.head(top_n_features)['Feature'].tolist()
    
    X_selected = X_cleaned[top_features].copy()
        
    mlflow.log_param("feature_selection_method", "RandomForest_Importance")
    mlflow.log_param("top_n_features", top_n_features)
    
    with open('rf_top_features.txt', 'w') as f:
        for feature in top_features:
            f.write(f"{feature}\n")
    
    feature_importance_df.to_csv('rf_feature_importance.csv', index=False)
    mlflow.log_artifact('rf_feature_importance.csv')
    mlflow.log_artifact('rf_top_features.txt')

🏃 View run RandomForest_Feature_Selection at: https://dagshub.com/dshan21/ML_ASS_2.mlflow/#/experiments/2/runs/29cfd337472944a7a3a6cc39f3b5a7e8
🧪 View experiment at: https://dagshub.com/dshan21/ML_ASS_2.mlflow/#/experiments/2


# MODEL TRAINING

In [13]:
import mlflow
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix

with mlflow.start_run(run_name="RandomForest_Training"):
    run_id = mlflow.active_run().info.run_id
    print(f"MLflow Run ID: {run_id}")
    
    params = {
        'n_estimators': 200,
        'max_depth': 5,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'max_features': 'sqrt',
        'bootstrap': True,
        'class_weight': 'balanced',
        'random_state': 42,
        'n_jobs': -1,
        'verbose': 1
    }
    
    for param, value in params.items():
        mlflow.log_param(param, value)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_selected, y, test_size=0.2, random_state=42, stratify=y
    )
    
    mlflow.log_param("num_features", X_selected.shape[1])
    mlflow.log_param("dataset_size", X_train.shape)
    mlflow.log_param("test_size", 0.2)
        
    rf = RandomForestClassifier(**params)
    rf.fit(X_train, y_train)
    
    best_iteration = params['n_estimators']
    
    y_pred_proba = rf.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_pred_proba)
    y_pred_binary = (y_pred_proba > 0.5).astype(int)
    
    
    mlflow.log_metric("auc", auc_score)
    mlflow.log_metric("best_iteration", best_iteration)
    
    mlflow.sklearn.log_model(rf,
        "random_forest_model",
        registered_model_name="FraudRandomForestModel")
    
    import joblib
    model_path = "random_forest_model.pkl"
    joblib.dump(rf, model_path)
    mlflow.log_artifact(model_path)
    
    feature_importance = rf.feature_importances_
    importance_df = pd.DataFrame({
        'Feature': X_selected.columns,
        'Importance': feature_importance
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df.head(20))
    plt.title('Top 20 Features by Importance')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()
    
    mlflow.log_artifact('feature_importance.png')
    
    cm = confusion_matrix(y_test, y_pred_binary)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('rf_confusion_matrix.png')
    plt.close()
    mlflow.log_artifact('rf_confusion_matrix.png')
    
    pipeline_config = {
        'model_type': 'RandomForest',
        'feature_selection': 'importance_based',
        'top_features': list(X_selected.columns),
        'preprocessing': 'Label encoding for categoricals, median imputation for numericals',
        'model_params': params,
        'auc_score': float(auc_score)
    }
    
    import json
    with open('pipeline_config.json', 'w') as f:
        json.dump(pipeline_config, f, indent=4)
    
    mlflow.log_artifact('pipeline_config.json')
    
    print(f"AUC Score: {auc_score:.4f}")

MLflow Run ID: 7d4866d62f864efab4f0ad9f68bd26cc


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   52.6s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   54.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.8s finished
2025/04/27 08:24:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'FraudRandomForestModel' already exists. Creating a new version of this model...
2025/04/27 08:24:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: FraudRandomFo

AUC Score: 0.8541
🏃 View run RandomForest_Training at: https://dagshub.com/dshan21/ML_ASS_2.mlflow/#/experiments/2/runs/7d4866d62f864efab4f0ad9f68bd26cc
🧪 View experiment at: https://dagshub.com/dshan21/ML_ASS_2.mlflow/#/experiments/2
